In [75]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import nltk
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\44783\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\44783\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [38]:
df_Fake_news = pd.read_csv('Fake.csv')
df_Real_news = pd.read_csv('True.csv')

In [39]:
df_Fake_news.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [40]:
df_Real_news.head() 

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [41]:
df_Fake_news["class"] = 0
df_Real_news["class"] = 1

In [43]:
df_Fake_news.shape, df_Real_news.shape

((23481, 5), (21417, 5))

In [45]:
df_Fake_news.shape, df_Real_news.shape

((23471, 5), (21407, 5))

In [46]:
#fake_manual_testing['class'] = 0
#real_manual_testing['class'] = 1

# Define the manual testing DataFrames
fake_news_manual_testing = df_Fake_news.tail(10).copy()
df_Fake_news = df_Fake_news.iloc[:-10]  # Keep all but the last 10 rows

real_news_manual_testing = df_Real_news.tail(10).copy()
df_Real_news = df_Real_news.iloc[:-10]  # Keep all but the last 10 rows





In [47]:
# Add 'class' column to the manual testing DataFrames
fake_news_manual_testing['class'] = 0
real_news_manual_testing['class'] = 1



In [48]:
# Optionally, concatenate these manual testing DataFrames if needed
manual_testing = pd.concat([fake_news_manual_testing, real_news_manual_testing])

# Display the manual_testing DataFrame to ensure everything is correct
#import ace_tools as tools; tools.display_dataframe_to_user(name="Manual Testing DataFrame", dataframe=manual_testing)



In [49]:
fake_news_manual_testing.head(10)

,title,text,subject,date,class
23461,REPORT: ‘Federal Government Escalated the Viol...,KILLED: Rancher and protest spokesman Robert ...,Middle-east,"January 28, 2016",0
23462,"BOILER ROOM – Oregon Standoff, Cuddle Parties,...",Tune in to the Alternate Current Radio Network...,Middle-east,"January 28, 2016",0
23463,"Eyewitness Says Feds Ambushed Bundys, 100 Shot...",Patrick Henningsen 21st Century Wire UPDATE: 1...,Middle-east,"January 27, 2016",0
23464,Episode #119 – SUNDAY WIRE: ‘You Know the Dril...,Episode #119 of SUNDAY WIRE SHOW finally resum...,Middle-east,"January 24, 2016",0
23465,‘There’ll be boots on the ground’: US making n...,21st Century Wire says Various parties in Wash...,Middle-east,"January 23, 2016",0
23466,Boston Brakes? How to Hack a New Car With Your...,21st Century Wire says For those who still ref...,Middle-east,"January 22, 2016",0
23467,Oregon Governor Says Feds ‘Must Act’ Against P...,"21st Century Wire says So far, after nearly 20...",Middle-east,"January 21, 2016",0
23468,Ron Paul on Burns Oregon Standoff and Jury Nul...,21st Century Wire says If you ve been followin...,Middle-east,"January 21, 2016",0
23469,BOILER ROOM: As the Frogs Slowly Boil – EP #40,Tune in to the Alternate Current Radio Network...,Middle-east,"January 20, 2016",0
23470,Arizona Rancher Protesting in Oregon is Target...,RTOne of the most visible members of the armed...,Middle-east,"January 20, 2016",0


In [50]:
real_news_manual_testing.head(10)

,title,text,subject,date,class
21397,Germany's Schulz says he would demand U.S. wit...,BERLIN (Reuters) - The leader of Germany s Soc...,worldnews,"August 23, 2017",1
21398,Blunt instrument? What a list of banned articl...,SHANGHAI (Reuters) - An old review of an acade...,worldnews,"August 23, 2017",1
21399,Saudi police release teenager detained for dan...,DUBAI (Reuters) - A 14-year-old boy who was de...,worldnews,"August 22, 2017",1
21400,"The People's Princess, Britons work to keep me...",LONDON (Reuters) - Abdul Daoud spilt most of t...,worldnews,"August 23, 2017",1
21401,"Argentina labor unions protest job losses, Mac...",BUENOS AIRES (Reuters) - Argentina s main labo...,worldnews,"August 22, 2017",1
21402,Exclusive: Trump's Afghan decision may increas...,ON BOARD A U.S. MILITARY AIRCRAFT (Reuters) - ...,worldnews,"August 22, 2017",1
21403,U.S. puts more pressure on Pakistan to help wi...,WASHINGTON (Reuters) - The United States sugge...,worldnews,"August 21, 2017",1
21404,Exclusive: U.S. to withhold up to $290 million...,WASHINGTON (Reuters) - The United States has d...,worldnews,"August 22, 2017",1
21405,Trump talks tough on Pakistan's 'terrorist' ha...,ISLAMABAD (Reuters) - Outlining a new strategy...,worldnews,"August 22, 2017",1
21406,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1


In [51]:
df_merge = pd.concat([df_Fake_news, df_Real_news], axis = 0)
df_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [52]:
df_newdata = df_merge.drop(['title','subject','date'], axis = 1)

In [53]:
df_newdata.isnull().sum()

text     0
class    0
dtype: int64

In [54]:
df_newdata = df_newdata.sample(frac = 1)

In [55]:
df_newdata.head()

,text,class
899,Rep. Adam Schiff (D-CA) is the Ranking Member ...,0
3309,Donald Trump is desperately trying to find A-l...,0
4202,A man almost lost his life on a flight out of ...,0
1905,HOUSTON (Reuters) - The U.S. Coast Guard on Mo...,1
6735,MOSCOW (Reuters) - Not only Russian President ...,1


In [56]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W", " ", text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape (string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text       

In [57]:
df_newdata['text'] = df_newdata['text'].apply(wordopt)

In [58]:
x = df_newdata['text']
y = df_newdata['class']

In [59]:
#x_train, x_text, y_train, y-test = train_text_split(x,y, test_size = 0.25)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)


In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the vectorizer
vectorization = TfidfVectorizer()

# Fit and transform the training data
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)


In [61]:
LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [62]:
pred_lr = LR.predict(xv_test)

In [63]:
LR.score(xv_test, y_test)

0.985911725367811

In [36]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5852
           1       0.99      0.98      0.99      5363

    accuracy                           0.99     11215
   macro avg       0.99      0.99      0.99     11215
weighted avg       0.99      0.99      0.99     11215



In [65]:
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [66]:
pred_dt = DT.predict(xv_test)

In [67]:
DT.score(xv_test, y_test)

0.9969683459652251

In [69]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5876
           1       1.00      1.00      1.00      5339

    accuracy                           1.00     11215
   macro avg       1.00      1.00      1.00     11215
weighted avg       1.00      1.00      1.00     11215



In [71]:
GB = GradientBoostingClassifier(random_state = 0)
GB.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [72]:
pred_gb = GB.predict(xv_test)

In [73]:
GB.score(xv_test, y_test)

0.996522514489523

In [74]:
print(classification_report(y_test, pred_gb))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5876
           1       0.99      1.00      1.00      5339

    accuracy                           1.00     11215
   macro avg       1.00      1.00      1.00     11215
weighted avg       1.00      1.00      1.00     11215



In [76]:
RF = RandomForestClassifier(random_state = 0)
RF.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [79]:
pred_rf = RF.predict(xv_test)

In [80]:
RF.score(xv_test, y_test)

0.9882300490414623

In [81]:
print(classification_report(y_test, pred_rf))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5876
           1       0.99      0.98      0.99      5339

    accuracy                           0.99     11215
   macro avg       0.99      0.99      0.99     11215
weighted avg       0.99      0.99      0.99     11215



In [99]:
# Sample data (replace with your actual data)
X = ["fake news example 1", "real news example 1", "fake news example 2", "real news example 2"]
y = [0, 1, 0, 1]

# Split the data
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the vectorizer
vectorization = TfidfVectorizer()

# Fit and transform the training data
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

# Define the models
LR = LogisticRegression()
DT = DecisionTreeClassifier()
GB = GradientBoostingClassifier()
RF = RandomForestClassifier()

# Train the models
LR.fit(xv_train, y_train)
DT.fit(xv_train, y_train)
GB.fit(xv_train, y_train)
RF.fit(xv_train, y_train)

# Function to convert model output to label
def output_label(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not Fake News"

# Function for manual testing
def manual_testing(news):
    # Transform the news data
    new_xv_test = vectorization.transform([news])

    # Make predictions with different models
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GB = GB.predict(new_xv_test)
    pred_RF = RF.predict(new_xv_test)

    # Print the predictions
    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGB Prediction: {} \nRF Prediction: {}".format(
        output_label(pred_LR[0]),
        output_label(pred_DT[0]),
        output_label(pred_GB[0]),
        output_label(pred_RF[0])
    ))

# Example usage
news = str(input("Enter news text: "))
manual_testing(news)


Enter news text: We all require joy, hope and optimism. In these darker times, take heart that people care, that kindnesses, innovation and generosity are happening all over the world. —  After the crushing and brutal massacre in San Bernardino, the terror attack in France and the shooting at a Colorado Planned Parenthood, we are in need of heartening news more than ever. Our souls require it to rehabilitate. It’s a time in our history when life can seem so out-of-control, when people appear more bent on vengeance than kindness and when they deem themselves entitled to take lives. It all coincides with our slow awakening of realizing the chasm of corruption is everywhere.  It can leave you disheartened, emotionally drifting and in such an overwhelmed state you may stand frozen because you have no idea of where to begin, or if any action you take will matter.  Despite the mass infusion of tragedy playing on every media, instances of generosity, heroism, hope, happiness, joy and love abo